api call

In [12]:
from openai import OpenAI
from tqdm import tqdm
import httpx
import pandas as pd
import openai
import backoff

api_key = "sk-KlLjOnG8myjg3Hhg5dF71699E6814e8b9753F00bB076C400"
client = OpenAI(
    base_url="https://oneapi.xty.app/v1",  # 中转url
    api_key=api_key,                      # api_key
    http_client=httpx.Client(
        proxies="http://127.0.0.1:7890"  # 代理地址
    ),
)

@backoff.on_exception(backoff.expo, (openai.APIStatusError, openai.InternalServerError), max_tries=5)
def get_responce(messages):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
    )
    return response.choices[0].message.content.strip('\n')


instruction_noindex_old = '''You will be provided with some log messages. You should check whether the giving log messages share the same template. If so, abstract variables with `{placeholders}` and return the template without additional explatation'''

instruction_noindex = '''You will be provided with some log messages. You should check if the giving log messages share the same template. If so, abstract possibily variables with `{{placeholders}}` to extract the corresponding template.
Print the input log's template delimited by backticks.
'''

instruction_100 = '''do the following log contents share the same template? Return the template or templates without additional explatation'''

instruction_1_13 = '''You will be provided with some log messages. You should check whether the giving log messages share the same template. If so, abstract variables with ‘{{placeholders}}’ and return the template without additional explatation'''

instruction_1_13_2 = '''Given some log templates, you should choice the most corret one. If you think none of them is correct, you can return the correct tempalte you think'''

prompt = '''Block broadcast_9 stored as values in memory (estimated size 8.8 KB, free 14.0 KB)'''

instruction_for_one_log = '''You will be provided with a log message delimited by backticks. You must abstract possibily variables with `{placeholders}` to extract the corresponding template.
Print the input log's template delimited by backticks.'''

input = '''Log message: `Block broadcast_9 stored as values in memory (estimated size 8.8 KB, free 14.0 KB)`'''
# ready=true,policy=3,wakefulness=1,wksummary=0x23,uasummary=0x1,bootcompleted=true,boostinprogress=false,waitmodeenable=false,mode=false,manual=38,auto=-1,adj=0.0userId=0
prompt2 = '''`normal`
`normal`
`normal`
`normal`
`normal`
`normal`
`normal`
`normal`
`normal`
`normal`
`normal`
`normal`
`normal`
`normal`
`normal`
`normal`'''

messages = [{"role": "system", "content": instruction_noindex}]
messages.append({"role": "user", "content": prompt2})
print(get_responce(messages))

`{{placeholder}}`


In [9]:
from collections import Counter


def most_common(lst):
    data = Counter(lst)
    return data.most_common(1)[0][0]


# 测试
lst = [1, 2, 7, 4, 3, 3]
print(most_common(lst))

3


Evaluate_gpt

In [27]:
from evaluator import evaluate
import pandas as pd
datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC',
            'Zookeeper', 'Mac', 'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']
datasets = ['HealthApp']
m,n,p,q = [],[],[],[]
for dataset in datasets:
    file = f'outputs/k_means/Second/{dataset}.csv'
    # df = pd.read_csv(f'outputs/k_means/initial/{dataset}.csv')
    # df2 =
    a,b,c,d = evaluate(file, dataset)
    m.append(a)
    n.append(b)
    p.append(c)
    q.append(d)

print(sum(m)/len(m))
print(sum(n)/len(n))

Precision: 0.7479, Recall: 0.9999, F1_measure: 0.8557, Group Accuracy: 0.6260, Message-Level Accuracy: 0.4870, Edit Distance: 11.5855
0.626
0.487


验证按' '分词有多少词，以及含数字的词占多少

In [15]:
import re
import pandas as pd
from post_process import correct_single_template
datasets = ['BGL' ,'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC', 'Zookeeper', 'Mac','Android','Hadoop', 'Windows', 'Apache', 'Thunderbird', 'Spark']
a = 0
b = 0
list = []
list2 = []
k = 1
temp = ['structured', 'templates', 'Content', 'EventTemplate']
tokens = []
for dataset in datasets:
    df = pd.read_csv(f"dataset\{dataset}\{dataset}_2k.log_{temp[k]}_corrected.csv")
    list = df[f'{temp[k + 2]}'].tolist()

    print('-' * 20)
    print(dataset)

    # 验证长度为1-3的tempalte是否含有<*>
    # NOTE: 长度为1时，不用解析，2-3仍然需要解析
    # DS
    # NOTE: 没有template中含有'  '
    # BL+US
    # NOTE: boolen变量主要出现在Android中 null:Mac, Android root:many datasets admin:OpenSSH, Thunderbird
    # BL = ['true', 'false']
    # US = ['null', 'root', 'admin']
    # DG
    # NOTE: 没有template含有纯数字
    for item in list:
        template = correct_single_template(item)
        if template != item:
            print(item)
            print(template)
            print('-' * 20)
    # 验证按' '分词有多少词，以及含数字的词占多少
    # list2.append(len(list))    
    # total_words = 0
    # words_with_numbers = 0

    # for s in list:
    #     words = s.split()
    #     total_words += len(words)
    #     words_with_numbers += len([word for word in words if re.search(r'\d', word)])
    # a+=total_words
    # b+=words_with_numbers


# print(f"{b} / {a}")

--------------------
BGL
--------------------
HDFS
--------------------
Linux
ALERT exited abnormally with [1]
ALERT exited abnormally with [<*>]
--------------------
ANONYMOUS FTP LOGIN FROM <*>,  (anonymous)
ANONYMOUS FTP LOGIN FROM <*>, (anonymous)
--------------------
apm: BIOS version <*>.<*> Flags <*> (Driver version <*>)
apm: BIOS version <*> Flags <*> (Driver version <*>)
--------------------
audit(<*>.<*>:<*>): initialized
audit(<*>): initialized
--------------------
authentication failure; logname= uid=0 euid=0 tty=:0 ruser= rhost=
authentication failure; logname= uid=<*> euid=<*> tty=:<*> ruser= rhost=
--------------------
authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=<*>
authentication failure; logname= uid=<*> euid=<*> tty=NODEVssh ruser= rhost=<*>
--------------------
authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=<*>  user=guest
authentication failure; logname= uid=<*> euid=<*> tty=NODEVssh ruser= rhost=<*> user=guest
--

In [21]:
import pandas as pd
df = pd.read_csv(f"dataset\Linux\Linux_2k.log_structured_corrected.csv")
list = df['EventTemplate'].tolist()
df = pd.read_csv(f"dataset\Linux\Linux_2k.log_templates_corrected.csv")
templates = df['EventTemplate'].tolist()
for item in list:
    if item not in templates:
        print(item)

authentication failure; logname= uid=<*> euid=<*> tty=<*> ruser= rhost=<*>
authentication failure; logname= uid=<*> euid=<*> tty=<*> ruser= rhost=<*>
authentication failure; logname= uid=<*> euid=<*> tty=<*> ruser= rhost=<*>  user=<*>
authentication failure; logname= uid=<*> euid=<*> tty=<*> ruser= rhost=<*>  user=<*>
authentication failure; logname= uid=<*> euid=<*> tty=<*> ruser= rhost=<*>  user=<*>
authentication failure; logname= uid=<*> euid=<*> tty=<*> ruser= rhost=<*>  user=<*>
authentication failure; logname= uid=<*> euid=<*> tty=<*> ruser= rhost=<*>  user=<*>
authentication failure; logname= uid=<*> euid=<*> tty=<*> ruser= rhost=<*>  user=<*>
authentication failure; logname= uid=<*> euid=<*> tty=<*> ruser= rhost=<*>  user=<*>
authentication failure; logname= uid=<*> euid=<*> tty=<*> ruser= rhost=<*>  user=<*>
authentication failure; logname= uid=<*> euid=<*> tty=<*> ruser= rhost=<*>  user=<*>
authentication failure; logname= uid=<*> euid=<*> tty=<*> ruser= rhost=<*>  user=<*>


In [2]:
import json
# load demonstrations
with open('demonstrations.json', 'r') as f:
    demonstrations = json.load(f)
messages = []
print(demonstrations['Spark'][0])

{'role': 'user', 'content': 'Log: Block broadcast_0 stored as values in memory (estimated size 384.0 B, free 317.5 KB)'}


聚类结果的自定义处理

In [22]:
import re

def replace_words(str):
    words = str.split(' ')
    print(words)
    for i in range(len(words)):
        if words[i].count('<*>') >= 2:
            if words[i].startswith('/'):
                temp = words[i][1:]
            # 保留前后的符号
            else:
                prefix = '' if not re.match(
                    r'^[\[\]\.\:\,\/\']', words[i]) else words[i][0]
                suffix = '' if not re.match(
                    r'.*[\[\]\.\:\,\/\']$', words[i]) else words[i][-1]
                temp = prefix + '<*>' + suffix
            str = str.replace(words[i], temp)
    return str

    # zookeeper
    # output_str = input_str.replace(' /<*>', ' <*>')

# 测试


inputs = '''Saved output of task ,attempt_<*>_m_<*>_r_<*>. to hdfs://<*>/pjhe/test/<*>/_temporary/0/task_<*>_m_<*>_r_<*>'''

lines = inputs.split('\n')
for line in lines:
    print(replace_words(line))

['Saved', 'output', 'of', 'task', ',attempt_<*>_m_<*>_r_<*>.', 'to', 'hdfs://<*>/pjhe/test/<*>/_temporary/0/task_<*>_m_<*>_r_<*>']
Saved output of task ,<*>. to <*>


In [33]:
from collections import Counter
list = ['1','1','1','3','3','3','4','4']
words = [word for word in list if re.search(r'\d', word)]
num = len(words)

freq = Counter(list)
print (len(freq))
print(freq.most_common(3))

3
[('1', 3), ('3', 3), ('4', 2)]


In [21]:
from openai import OpenAI
import httpx
client = OpenAI(
    base_url="https://4.0.996444.icu/v1", 
    api_key="sk-1XhqpmIwxu9nCuCD51FcA00948F54eDf9cA4765998Ce8f0d",
    http_client=httpx.Client(
        proxies="http://127.0.0.1:7890"  # 代理地址
    ),
)
message_list = [
     {"role": "user", "content": "Hello"}, 
    ]

response = client.chat.completions.create(
    model="gpt-4",
    messages=message_list,
    temperature=0,
    max_tokens=1024
)

print(response)
response = response.choices[0].message.content

ChatCompletion(id='chatcmpl-8gsegeGGp7R5X0cjWY4oLnFg1Rh66', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1705230562, model='gpt-4', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=8, total_tokens=17))


Adjusted function

In [27]:
import re
def correct_single_template(template, user_strings=None):
    """Apply all rules to process a template.

    DS (Double Space)
    BL (Boolean)
    US (User String)
    DG (Digit)
    PS (Path-like String)
    WV (Word concatenated with Variable)
    DV (Dot-separated Variables)
    CV (Consecutive Variables)

    """

    boolean = {'true', 'false'}
    default_strings = {'null', 'root', 'admin'}
    path_delimiters = {  # reduced set of delimiters for tokenizing for checking the path-like strings
        r'\s', r'\,', r'\!', r'\;', r'\:',
        r'\=', r'\|', r'\"', r'\'',
        r'\[', r'\]', r'\(', r'\)', r'\{', r'\}'
    }
    token_delimiters = path_delimiters.union({  # all delimiters for tokenizing the remaining rules
        r'\.', r'\-', r'\+', r'\@', r'\#', r'\$', r'\%', r'\&',
    })

    if user_strings:
        default_strings = default_strings.union(user_strings)

    # apply DS
    template = template.strip()
    template = re.sub(r'\s+', ' ', template)

    r'\{\{.*?\}\}'

    # apply PS (Path-like String)
    # p_tokens = re.split('(' + '|'.join(path_delimiters) + ')', template)
    # new_p_tokens = []
    # for p_token in p_tokens:
    #     if re.match(r'^(\/[^\/]+)+$', p_token):
    #         p_token = '<*>'
    #     new_p_tokens.append(p_token)
    # template = ''.join(new_p_tokens)

    # tokenize for the remaining rules
    # tokens = re.split('(' + '|'.join(token_delimiters) + ')',
    #                   template)  # tokenizing while keeping delimiters
    new_tokens = []
    for token in tokens:
        # apply BL, US
        # for to_replace in boolean.union(default_strings):
        #     if token.lower() == to_replace.lower():
        #         token = '<*>'

        # apply DG
        # if re.match(r'^\d+$', token):
        #     token = '<*>'

        # apply WV
        if re.match(r'^[^\s\/]*<\*>[^\s\/]*$', token):
            if token != '<*>/<*>':  # need to check this because `/` is not a deliminator
                token = '<*>'

        # collect the result
        new_tokens.append(token)

    # make the template using new_tokens
    template = ''.join(new_tokens)

    # Substitute consecutive variables only if separated with any delimiter including "." (DV)
    while True:
        prev = template
        template = re.sub(r'<\*>\.<\*>', '<*>', template)
        if prev == template:
            break

    # Substitute consecutive variables only if not separated with any delimiter including space (CV)
    # NOTE: this should be done at the end
    while True:
        prev = template
        template = re.sub(r'<\*><\*>', '<*>', template)
        if prev == template:
            break

    while "#<*>#" in template:
        template = template.replace("#<*>#", "<*>")

    while "<*>:<*>" in template:
        template = template.replace("<*>:<*>", "<*>")
    return template

def parse_batch_log_template(response, logs=None):
    if "\n\n" in response:
        response = response.split("\n\n")[0]
    reg = re.compile("`([^`]+)`")
    tmps = reg.findall(response)
    tmps = [x.strip('\n').strip() for x in tmps]
    if len(tmps) == 0:
        return ''
    elif len(tmps) == 1:
        return tmps[0]
    else:
        return max(tmps, key=len)


# `updateLightsLocked,mInCall ={boolean},mScreenOn = {boolean},ledNotification == null?{boolean}`

input = '''`Finished task {{task_id}} in stage {{stage_id}} (TID {{task_tid}}). {{bytes}} bytes result sent to driver`'''
print(correct_single_template(parse_batch_log_template(input)))
# parse_batch_log_template(input)

Finished task {{task_id}} in stage {{stage_id}} (TID {{task_tid}}). {{bytes}} bytes result sent to driver


In [30]:
re.sub(r'(?i)(?<![a-z])true(?![a-z])', '<*>', ':True', flags=re.IGNORECASE)

':<*>'